In [5]:
#importing modules/libraries
import cv2
import mediapipe as mp
import numpy as np
import math as m
import csv

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [7]:
#calculating angle between two lines joing three major joints
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [8]:
#Colors.
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)
white= (255, 255, 255)

In [10]:

#capturing Video
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 10000)  #1280
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 10000) #720
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
result = cv2.VideoWriter('Posture.mp4', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         15, size)
# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    k=0
    while cap.isOpened():
        ret, frame = cap.read()
              
        # Get fps.
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        # Get height and width.
        h, w = frame.shape[:2]        
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            #left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
            #right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
            left_foot_index= [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            right_foot_index= [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            #getting exact cordinate locatons in integer values
            left_hip_int=(int(left_hip[0]*w),int(left_hip[1]*h))
            right_hip_int=(int(right_hip[0]*w),int(right_hip[1]*h))
            left_knee_int=(int(left_knee[0]*w),int(left_knee[1]*h))
            right_knee_int=(int(right_knee[0]*w),int(right_knee[1]*h))
            left_ankle_int=(int(left_ankle[0]*w),int(left_ankle[1]*h))
            right_ankle_int=(int(right_ankle[0]*w),int(right_ankle[1]*h))
            left_foot_index_int=(int(left_foot_index[0]*w),int(left_foot_index[1]*h))
            right_foot_index_int=(int(right_foot_index[0]*w),int(right_foot_index[1]*h))
            
            
            
            # Calculate angle
            angle_lknee = int(calculate_angle(left_hip, left_knee, left_ankle))
            angle_lhip= int(calculate_angle(left_shoulder,left_hip,left_knee))
            angle_lankle= int(calculate_angle(left_knee,left_ankle,left_foot_index))
            angle_rknee = int(calculate_angle(right_hip, right_knee, right_ankle))
            angle_rhip= int(calculate_angle(right_shoulder,right_hip,right_knee))
            angle_rankle= int(calculate_angle(right_knee,right_ankle,right_foot_index))
            

            # Visualizing angles on the specific body joint
            cv2.putText(image, str(angle_lknee),left_knee_int,cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, 2, cv2.LINE_AA)
            cv2.putText(image, str(angle_lhip),left_hip_int,cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, 2, cv2.LINE_AA)
            cv2.putText(image, str(angle_lankle),left_ankle_int,cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, 2, cv2.LINE_AA)   
            cv2.putText(image, str(angle_rknee),right_knee_int,cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, 2, cv2.LINE_AA)
            cv2.putText(image, str(angle_rhip),right_hip_int,cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, 2, cv2.LINE_AA)
            cv2.putText(image, str(angle_rankle),right_ankle_int,cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, 2, cv2.LINE_AA)   
                       
        except:
            pass
        if k==0:
            l = [angle_lknee,angle_lhip,angle_lankle,angle_rknee,angle_rhip,angle_rankle]
        else:
            l.append([angle_lknee,angle_lhip,angle_lankle,angle_rknee,angle_rhip,angle_rankle])
               
        #pointing the landmark(body joint)
        cv2.circle(image, (left_hip_int), 4, yellow, -1)
        cv2.circle(image, (right_hip_int), 4, yellow, -1)
        cv2.circle(image, (left_knee_int), 4, yellow, -1)
        cv2.circle(image, (right_knee_int), 4, yellow, -1)
        cv2.circle(image, (left_ankle_int), 4, yellow, -1)
        cv2.circle(image, (right_ankle_int), 4, yellow, -1)
        cv2.circle(image, (left_foot_index_int), 4, yellow, -1)
        cv2.circle(image, (right_foot_index_int), 4, yellow, -1)
        
        # Joining landmarks
        cv2.line(image, (left_hip_int), (right_hip_int), green, 2)
        cv2.line(image, (left_hip_int), (left_knee_int), green, 2)
        cv2.line(image, (right_hip_int), (right_knee_int), green, 2)
        cv2.line(image, (left_knee_int), (left_ankle_int), green, 2)        
        cv2.line(image, (right_knee_int), (right_ankle_int), green, 2)
        cv2.line(image, (left_ankle_int), (left_foot_index_int), green, 2)        
        cv2.line(image, (right_ankle_int), (right_foot_index_int), green, 2)                  
        
        
        cv2.imshow('Mediapipe Feed', image)  
        result.write(image)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
        k+=1    
    cap.release()
    result.release()
    cv2.destroyAllWindows()
 

In [11]:
l

[179,
 175,
 170,
 179,
 177,
 171,
 [178, 175, 169, 179, 176, 170],
 [178, 175, 168, 179, 176, 170],
 [178, 175, 168, 179, 176, 170],
 [179, 176, 167, 179, 175, 171],
 [178, 176, 167, 179, 175, 171],
 [178, 176, 168, 178, 175, 170],
 [177, 175, 170, 178, 176, 171],
 [178, 175, 172, 179, 176, 171],
 [178, 175, 172, 179, 177, 170],
 [178, 175, 171, 179, 177, 171],
 [178, 175, 174, 179, 177, 170],
 [176, 176, 174, 178, 177, 170],
 [176, 175, 174, 178, 177, 170],
 [177, 175, 173, 179, 177, 170],
 [178, 176, 171, 179, 175, 170],
 [178, 177, 171, 179, 174, 170],
 [178, 177, 170, 179, 174, 170],
 [178, 177, 170, 178, 174, 170]]